In [1]:
import os.path as osp

ROOT_DIR='./'
CKPT_PATH=osp.join(ROOT_DIR,'checkpoints/EffB5FaceAcc0.9841.ckpt')
CFG_PATH=osp.join(ROOT_DIR,'configs/faceb5.yml')

In [2]:
# ensure that data is valid, please wait for a while
from preprocessing.sanity_checks import check_all
check_all(osp.join(ROOT_DIR,"data/train/"))
check_all(osp.join(ROOT_DIR,"data/public/"))

INFO:root:Checking video ids in generated labels. Step 1/3
INFO:root:Checking labels in generated labels with original labels. Step 2/3
INFO:root:Checking labels in generated labels with existing keyframes / bounding boxes. Step 3/3
INFO:root:Checking video ids in generated labels. Step 1/3
INFO:root:Checking labels in generated labels with original labels. Step 2/3
INFO:root:Checking labels in generated labels with existing keyframes / bounding boxes. Step 3/3


## Validate your model on the test set (image level)

In [3]:
!python $ROOT_DIR/scripts/validate.py -c $CFG_PATH --o global.pretrained=$CKPT_PATH

Overriding configurating
Loaded pretrained weights for efficientnet-b5
Loaded pretrained weights for efficientnet-b5
/home/nhtlong/miniconda3/envs/zaloai/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Overriding configurating
Loaded pretrained weights for efficientnet-b5
Loaded pretrained weights for efficientnet-b5
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
-----------------------------------------------------------------------

## Validate your model on the test set (video level)

In [9]:
VAL_CFG_PATH = osp.join(ROOT_DIR,'configs/validation/faceb5.yml')
!CUDA_VISIBLE_DEVICES=0,1 python scripts/predict.py -c  $VAL_CFG_PATH \
                                                    -o  global.pretrained=$CKPT_PATH \
                                                        global.save_path="results/val.csv"

Overriding configurating
/home/nhtlong/miniconda3/envs/zaloai/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
Overriding configurating
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

LOCAL_RANK: 1 - CUDA_VISIBLE_DEVICES: [0,1]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Predicting DataLoader 0: 100%|████████████████| 140/140 [00:55<00

In [11]:
!python competition/strategy/confident.py --input="results/val.csv" --output="results/val_video.csv"

59


In [14]:
!python competition/test.py --pred="results/val_video.csv" --gt="data/train/labels_video_test.csv"

╒═══════════════════════╤═════════════════════════╕
│   number of gt sample │   number of predictions │
╞═══════════════════════╪═════════════════════════╡
│                    59 │                      59 │
╘═══════════════════════╧═════════════════════════╛
╒═══════╤═══════╤═══════╕
│   fpr │   fnr │   eer │
╞═══════╪═══════╪═══════╡
│     0 │     0 │     0 │
╘═══════╧═══════╧═══════╛
╒═════════════╤═══════╕
│   threshold │   acc │
╞═════════════╪═══════╡
│         0.5 │     1 │
╘═════════════╧═══════╛


## Predict on the test set 

In [10]:
PUBLIC_CFG_PATH = osp.join(ROOT_DIR,'configs/public/faceb5.yml')
!CUDA_VISIBLE_DEVICES=0,1 python scripts/predict.py -c  $PUBLIC_CFG_PATH \
                                                    -o  global.pretrained=$CKPT_PATH \
                                                        global.save_path="results/public.csv"

Overriding configurating
/home/nhtlong/miniconda3/envs/zaloai/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Overriding configurating
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

LOCAL_RANK: 1 - CUDA_VISIBLE_DEVICES: [0,1]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Predicting DataLoader 0: 100%|████████████████| 822/822 [05:07<00

In [16]:
!python competition/strategy/confident.py --input="results/public.csv" --output="results/submission.csv"

350
